In [1]:
from jetcam.csi_camera import CSICamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mping
kenel_size=5
low_threshold=50
high_threshold=200
def draw_lines(image,lines, color=[255,0,0],thickness=5):
    for line in lines:
        for x1,y1,x2,y2 in lines:
            cv2.line(image,(x1, y1), (x2, y2), color, thickness)

def hough_lines(image):
    lines=cv2.HoughLinesP(image,rho=2,theta=np.pi/180,threshold=90,np.array([]),minLineLength=120,maxLineGap=150)
    line_image=np.zeros((image.shape[0],image.shape[1]m3),dtype=np.uint8)
    draw_lines(line_image,lines)
    return line_image
#masked function
def region_of_interest(image,vertices):
    mask= np.zeros_like(image)
    if len(image.shape) >2:
        channel_count = image.shape[2]
        ignore_mask_color =(255,)*channel_count
    else:
        ignore_mask_color=255

    cv2.fillpoly(mask,vertices, ignore_mask_color)
    masked_image=cv2.bitwise_and(image,mask)
    return masked_image

camera = CSICamera(width=448, height=448)

image = camera.read()

camera.running = True
#coding
def update_image(change):
    image_initial = change['new']
    image_widget.value = bgr8_to_jpeg(image_initial)#이 밑에부터 차선 코딩
    image=cv2.cvtColor(image_initial,cv2.COLOR_RGB2GRAY)# gray process
    image= cv2.GaussianBlur(image,(kernel_size,kernel_size),0)# kernelsize 입력
    image=cv2.Canny(image,low_threshold,high_threshold)#edge detect
    imshape=image.shape
    vertices= np.array([[(100,imshape[0]),(450,320),(550,320),(imshape[1]-20,imshape[0])]],dtype=np.int32)
    mask=region_of_interest(image,vertices)
    lines=hough_lines(mask)
    lines_edges=cv2.addWeighted(image_initial,a=0.8,lines,b=1,r=0)
    #load center and car center  해야됨
    x_left=[448*223+223:448*223]
    x_left=lis.index(x_left,[255,255,255])
    x_right=[448*223+223:448*224-1]
    x_right=lis.index(x_right,[255,255,255])
    #autonomous driving
    load_center=[(x_right+x_left)/2,y] #load center
    camera_center=[224,y] #앞이 x라고 가정 (반대였던거같기도)
    #이 점들을 어느 image에 그릴지 정해야됨.
    cv2.circle(image.camera_center,radius=3,color=[255,255,0],thickness=1)#mark the point in the center of the camera
    cv2.circle(image.load_center.radius=3,color=[255,255,0],thickness=1)#mark the point in the center of load

    seta=atan((load_center-camera_center)/y)
    car.steering=seta
    
    camera.observe(update_image, names='value')

display(image_widget)

NameError: name 'camera_center' is not defined